In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from glob import glob 
from PIL import Image, ImageOps
import albumentations as A
import cv2
from tqdm import tqdm
import random

In [3]:
# 시드 고정
random_seed = 42
random.seed(random_seed)  # Python의 내장 random 모듈의 시드 고정
np.random.seed(random_seed)  # NumPy의 시드 고정
os.environ['PYTHONHASHSEED'] = str(random_seed)  # 환경 변수를 통한 Python 해시 시드 고정

In [4]:
# 데이터셋 로드
df = pd.read_csv('./data/train.csv')

In [5]:
# 증강 파이프라인 정의
transforms = A.Compose([
    A.OneOf([
        A.HorizontalFlip(),  # 이미지를 수평으로 뒤집습니다.
        A.VerticalFlip(),  # 이미지를 수직으로 뒤집습니다.
    ], p=1),  # 'OneOf'는 괄호 안의 변환 중 하나(수평 뒤집기와 수직 뒤집기 중 하나)를 무작위로 선택하여 적용합니다.
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=0.5),  # 이미지의 밝기와 대비를 무작위로 조절합니다. 'brightness_limit'와 'contrast_limit'는 변형의 강도를 결정합니다.
    A.HueSaturationValue(p=0.5),  # 이미지의 색조(Hue), 채도(Saturation), 밝기(Value)를 무작위로 변경합니다. 이를 통해 색상 변형을 추가합니다.
    A.Transpose(p=0.5),  # 이미지를 전치합니다. 즉, 이미지의 가로와 세로를 바꾸는 변환을 적용합니다.
    A.Rotate(limit=90, border_mode=cv2.BORDER_REPLICATE),  # 이미지를 최대 90도까지 무작위로 회전합니다. 'border_mode'는 회전 시 테두리를 어떻게 처리할지 결정합니다.
    A.CoarseDropout(p=0.5, max_holes=30, max_height=8, max_width=8, min_holes=8, min_height=8, min_width=8),  # 이미지에 무작위로 구멍(드롭아웃)을 생성합니다. 구멍의 수와 크기는 지정된 범위 내에서 무작위로 결정됩니다.
    A.ElasticTransform(p=0.5, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(0, 0, 0), mask_value=None, approximate=False),  # 탄성 변형을 적용하여 이미지에 왜곡을 추가합니다. 이 변형은 이미지를 유연하게 휘게 만듭니다.
    A.SomeOf([
        A.Blur(blur_limit=3),  # 이미지를 흐리게 만듭니다. 'blur_limit'는 흐림 효과의 강도를 결정합니다.
        A.MotionBlur(blur_limit=3),  # 이미지에 움직임 흐림(motion blur) 효과를 추가합니다.
        A.Downscale(scale_min=0.699999988079071, scale_max=0.9900000095367432, interpolation=3),  # 이미지의 해상도를 낮춥니다.
        A.GaussNoise(var_limit=(0, 700), per_channel=True),  # 이미지에 가우시안 노이즈를 추가합니다.
    ], n=2, p=0.85),  # 'SomeOf'는 괄호 안의 변환 중 지정된 수(n=2)만큼을 무작위로 선택하여 적용합니다.
    A.GridDistortion(p=0.5, num_steps=5, distort_limit=(-0.029999999329447746, 0.05000000074505806), interpolation=2, border_mode=0, value=(0, 0, 0), mask_value=None, normalized=True)  # 이미지에 그리드 왜곡 효과를 적용합니다. 이 변형은 이미지를 그리드로 나누고 각 그리드 포인트를 무작위로 이동시켜 이미지에 왜곡을 추가합니다. num_steps는 그리드의 크기를 결정하며, distort_limit는 왜곡의 강도를 조절합니다.
])

In [6]:
data_path = './data/'
train_path = os.path.join(data_path, "train")  # train 이미지가 저장된 경로

# 'augmented/' 폴더가 없으면 생성
augmented_folder_path = os.path.join(data_path, "augmented")
os.makedirs(augmented_folder_path, exist_ok=True)

# 'augmented/A' 폴더가 없으면 생성
albumentations_path = os.path.join(augmented_folder_path, "A")
os.makedirs(albumentations_path, exist_ok=True)

ids = []
targets = []

for index, ID, target in tqdm(df.itertuples(), desc='Image augmentation', mininterval=0.1):
    image_path = os.path.join(train_path, ID)  # 수정된 경로
    image = np.array(Image.open(image_path))
    if target == 13:
        n = 25
    elif target == 14:
        n = 35
    elif target == 1:
        n = 45
    else:
        n = 20
    for i in range(n):
        transformed_image = transforms(image=image)['image']
        image_ID = f'tf{i}_' + ID 
        ids.append(image_ID)
        targets.append(target)
        # 증강된 이미지를 'augmented/A' 폴더에 저장
        augmented_image_path = os.path.join(albumentations_path, image_ID)
        Image.fromarray(transformed_image).save(augmented_image_path)

# 증강된 이미지 데이터를 DataFrame에 추가하고 결과 저장
aug_data = {'ID': ids, 'target': targets}
aug_df = pd.DataFrame(aug_data)    

Image augmentation: 0it [00:00, ?it/s]

Image augmentation: 1570it [55:30,  2.12s/it]


In [7]:
# 원본 데이터셋의 크기
original_size = len(df)
# 증강된 데이터셋의 크기
augmented_size = len(aug_df)
# 전체 데이터셋의 크기
total_size = len(df) + len(aug_df)

print(f"원본 데이터셋 크기: {original_size}")
print(f"증강된 데이터셋 크기: {augmented_size}")
print(f"전체 데이터셋 크기: {total_size}")

원본 데이터셋 크기: 1570
증강된 데이터셋 크기: 33670
전체 데이터셋 크기: 35240


In [8]:
# 'augmented/albumentations' 폴더 내의 파일 수 확인
augmented_files = os.listdir(albumentations_path)
print(f"'A' 폴더 내의 파일 수: {len(augmented_files)}")

'A' 폴더 내의 파일 수: 33670


In [9]:
aug_df.to_csv('./data/A_train.csv', index=False)

In [10]:
# import shutil
# import os

# # 증강된 이미지가 저장된 폴더 삭제
# albumentations_path = './data/augmented/A'
# if os.path.exists(albumentations_path):
#     shutil.rmtree(albumentations_path)
#     print(f"'{albumentations_path}' 폴더와 그 안의 모든 이미지가 삭제되었습니다.")

# # 생성된 CSV 파일 삭제
# csv_file_path = './data/A_train.csv'
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_path)
#     print(f"'{csv_file_path}' 파일이 삭제되었습니다.")
